### Dowload e Leitura do Ames Dataset

In [4]:
import pathlib
import pickle
import requests
import pandas as pd

In [ ]:
DATA_DIR = pathlib.Path.cwd().parent / 'data'
print(DATA_DIR)

DATA_DIR.mkdir(parents=True, exist_ok=True)

c:\Users\giuli\OneDrive - Insper - Institudo de Ensino e Pesquisa\Documentos\insper\MachineLearning\ProjetoAmes-ML\data


##### *Download do dataset*

Neste passo, realizaremos o download do dataset, a leitura dos dados e checagem das colunas e dos tipos de dados presentes no dataset. Assim, poderemos corrigir seus tipos no pandas. 

In [7]:
raw_data_dir = DATA_DIR / 'raw'
raw_data_dir.mkdir(parents=True, exist_ok=True)
print(raw_data_dir)

raw_data_file_path = DATA_DIR / 'raw' / 'ames.csv'
print(raw_data_file_path)

if not raw_data_file_path.exists():
    source_url = 'https://www.openintro.org/book/statdata/ames.csv'
    headers = {
        'User-Agent': \
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) ' \
            'AppleWebKit/537.36 (KHTML, like Gecko) ' \
            'Chrome/39.0.2171.95 Safari/537.36',
    }
    response = requests.get(source_url, headers=headers)
    csv_content = response.content.decode()
    with open(raw_data_file_path, 'w', encoding='utf8') as file:
        file.write(csv_content)

c:\Users\giuli\OneDrive - Insper - Institudo de Ensino e Pesquisa\Documentos\insper\MachineLearning\data\raw
c:\Users\giuli\OneDrive - Insper - Institudo de Ensino e Pesquisa\Documentos\insper\MachineLearning\data\raw\ames.csv


##### *Leitura do dataset*

A seguir, vamos realizar algumas certificações de que o download foi bem sucedido e que os dados estão corretos. Então podemos verificar o tamanho do dataset, a quantidade de linhas e colunas e as primeiras linhas do dataset.

In [9]:
filesize = raw_data_file_path.stat().st_size
print(f'This file has {filesize} bytes')

This file has 1185234 bytes


In [10]:
raw_data = pd.read_csv(raw_data_file_path)

In [11]:
raw_data.shape

(2930, 82)

In [12]:
raw_data.head()

,Order,PID,MS.SubClass,MS.Zoning,Lot.Frontage,Lot.Area,Street,Alley,Lot.Shape,Land.Contour,...,Pool.Area,Pool.QC,Fence,Misc.Feature,Misc.Val,Mo.Sold,Yr.Sold,Sale.Type,Sale.Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


A partir deste momento, `raw_data` se torna o dataset original e não será mais alterado. Assim, vamos realizar uma cópia do dataset para realizar as alterações necessárias.

In [13]:
data = raw_data.copy()

In [14]:
data.dtypes.value_counts()

object     43
int64      28
float64    11
Name: count, dtype: int64

##### *Transformação dos dados categóricos em numéricos*

Neste passo, é necessário acessar a documentação do dataset para entender o significado de cada coluna e o que ela representa. Após a análise da documentação, podemos dividir as colunas dado o seu tipo de dado e o que ela representa. Além disso, devemos ignorar colunas que não são relevantes para a análise, como o ID do imóvel.

In [15]:
ignore_variables = [
    'Order',
    'PID',
]

continuous_variables = [
    'Lot.Frontage',
    'Lot.Area',
    'Mas.Vnr.Area',
    'BsmtFin.SF.1',
    'BsmtFin.SF.2',
    'Bsmt.Unf.SF',
    'Total.Bsmt.SF',
    'X1st.Flr.SF',
    'X2nd.Flr.SF',
    'Low.Qual.Fin.SF',
    'Gr.Liv.Area',
    'Garage.Area',
    'Wood.Deck.SF',
    'Open.Porch.SF',
    'Enclosed.Porch',
    'X3Ssn.Porch',
    'Screen.Porch',
    'Pool.Area',
    'Misc.Val',
    'SalePrice',
]

discrete_variables = [
    'Year.Built',
    'Year.Remod.Add',
    'Bsmt.Full.Bath',
    'Bsmt.Half.Bath',
    'Full.Bath',
    'Half.Bath',
    'Bedroom.AbvGr',
    'Kitchen.AbvGr',
    'TotRms.AbvGrd',
    'Fireplaces',
    'Garage.Yr.Blt',
    'Garage.Cars',
    'Mo.Sold',
    'Yr.Sold',
]

ordinal_variables = [
    'Lot.Shape',
    'Utilities',
    'Land.Slope',
    'Overall.Qual',
    'Overall.Cond',
    'Exter.Qual',
    'Exter.Cond',
    'Bsmt.Qual',
    'Bsmt.Cond',
    'Bsmt.Exposure',
    'BsmtFin.Type.1',
    'BsmtFin.Type.2',
    'Heating.QC',
    'Electrical',
    'Kitchen.Qual',
    'Functional',
    'Fireplace.Qu',
    'Garage.Finish',
    'Garage.Qual',
    'Garage.Cond',
    'Paved.Drive',
    'Pool.QC',
    'Fence',
]

categorical_variables = [
    'MS.SubClass',
    'MS.Zoning',
    'Street',
    'Alley',
    'Land.Contour',
    'Lot.Config',
    'Neighborhood',
    'Condition.1',
    'Condition.2',
    'Bldg.Type',
    'House.Style',
    'Roof.Style',
    'Roof.Matl',
    'Exterior.1st',
    'Exterior.2nd',
    'Mas.Vnr.Type',
    'Foundation',
    'Heating',
    'Central.Air',
    'Garage.Type',
    'Misc.Feature',
    'Sale.Type',
    'Sale.Condition',
]

In [ ]:
data.drop(columns=['Order', 'PID'], inplace=True)

Agora vamos fazer uma primeira tentativa de corrigir os tipos de dados das variáveis dadas, definindo os tipos da seguinte maneira:

- contínuas: float64
- categóricas: category
- ordinais: Este é o mais difícil, eles têm que ser do tipo category, mas precisamos definir a ordem. Um por um.
- discretas: Vamos defini-las como sendo do tipo float64, ou seja, vamos interpretá-las como quantidades numéricas.

In [16]:
for col in continuous_variables:
    data[col] = data[col].astype('float64')

In [17]:
for col in categorical_variables:
    data[col] = data[col].astype('category')

In [18]:
for col in discrete_variables:
    data[col] = data[col].astype('float64')

In [19]:
data[ordinal_variables].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Lot.Shape       2930 non-null   object
 1   Utilities       2930 non-null   object
 2   Land.Slope      2930 non-null   object
 3   Overall.Qual    2930 non-null   int64 
 4   Overall.Cond    2930 non-null   int64 
 5   Exter.Qual      2930 non-null   object
 6   Exter.Cond      2930 non-null   object
 7   Bsmt.Qual       2850 non-null   object
 8   Bsmt.Cond       2850 non-null   object
 9   Bsmt.Exposure   2847 non-null   object
 10  BsmtFin.Type.1  2850 non-null   object
 11  BsmtFin.Type.2  2849 non-null   object
 12  Heating.QC      2930 non-null   object
 13  Electrical      2929 non-null   object
 14  Kitchen.Qual    2930 non-null   object
 15  Functional      2930 non-null   object
 16  Fireplace.Qu    1508 non-null   object
 17  Garage.Finish   2771 non-null   object
 18  Garage.Q

In [20]:
category_orderings = {
    'Lot.Shape': [
        'Reg',
        'IR1',
        'IR2',
        'IR3',
    ],
    'Utilities': [
        'AllPub',
        'NoSewr',
        'NoSeWa',
        'ELO',
    ],
    'Land.Slope': [
        'Gtl',
        'Mod',
        'Sev',
    ],
    'Overall.Qual': [
        1,
        2,
        3,
        4,
        5,
        6,
        7,
        8,
        9,
        10,
    ],
    'Overall.Cond': [
        1,
        2,
        3,
        4,
        5,
        6,
        7,
        8,
        9,
        10,
    ],
    'Exter.Qual': [
        'Ex',
        'Gd',
        'TA',
        'Fa',
        'Po',
    ],
    'Exter.Cond': [
        'Ex',
        'Gd',
        'TA',
        'Fa',
        'Po',
    ],
    'Bsmt.Qual': [
        'Ex',
        'Gd',
        'TA',
        'Fa',
        'Po',
    ],
    'Bsmt.Cond': [
        'Ex',
        'Gd',
        'TA',
        'Fa',
        'Po',
    ],
    'Bsmt.Exposure': [
        'Gd',
        'Av',
        'Mn',
        'No',
        'NA',
    ],
    'BsmtFin.Type.1': [
        'GLQ',
        'ALQ',
        'BLQ',
        'Rec',
        'LwQ',
        'Unf',
    ],
    'BsmtFin.Type.2': [
        'GLQ',
        'ALQ',
        'BLQ',
        'Rec',
        'LwQ',
        'Unf',
    ],
    'Heating.QC': [
        'Ex',
        'Gd',
        'TA',
        'Fa',
        'Po',
    ],
    'Electrical': [
        'SBrkr',
        'FuseA',
        'FuseF',
        'FuseP',
        'Mix',
    ],
    'Kitchen.Qual': [
        'Ex',
        'Gd',
        'TA',
        'Fa',
        'Po',
    ],
    'Functional': [
        'Typ',
        'Min1',
        'Min2',
        'Mod',
        'Maj1',
        'Maj2',
        'Sev',
        'Sal',
    ],
    'Fireplace.Qu': [
        'Ex',
        'Gd',
        'TA',
        'Fa',
        'Po',
    ],
    'Garage.Finish': [
        'Fin',
        'RFn',
        'Unf',
    ],
    'Garage.Qual': [
        'Ex',
        'Gd',
        'TA',
        'Fa',
        'Po',
    ],
    'Garage.Cond': [
        'Ex',
        'Gd',
        'TA',
        'Fa',
        'Po',    
    ],
    'Paved.Drive': [
        'Y',
        'P',
        'N',
    ],
    'Pool.QC': [
        'Ex',
        'Gd',
        'TA',
        'Fa',
    ],
    'Fence': [
        'GdPrv',
        'MnPrv',
        'GdWo',
        'MnWw',
    ],
}

In [24]:
for col, orderings in category_orderings.items():
    data[col] = data[col] \
        .astype('category') \
        .cat \
        .set_categories(orderings, ordered=True)

Agora que nossas variáveis estão cuidadosamente organizadas, com os tipos adequados, vamos ver um resumo de cada uma.
Abaixo, podemos observar a quantidade de respostas para cada uma das variáveis categóricas do dataset.

In [25]:
data \
    .select_dtypes('category') \
    .describe() \
    .transpose() \
    .sort_values(by='count', ascending=True)

,count,unique,top,freq
Pool.QC,13,4,Ex,4
Misc.Feature,106,5,Shed,95
Alley,198,2,Grvl,120
Fence,572,4,MnPrv,330
Mas.Vnr.Type,1155,4,BrkFace,880
Fireplace.Qu,1508,5,Gd,744
Garage.Cond,2771,5,TA,2665
Garage.Qual,2771,5,TA,2615
Garage.Finish,2771,3,Unf,1231
Garage.Type,2773,6,Attchd,1731


Já o código abaixo mostra a quantidade de categorias que existem para cada coluna categórica do dataset.

In [22]:
data \
    .select_dtypes('category') \
    .describe() \
    .transpose() \
    .sort_values(by='unique', ascending=True)

,count,unique,top,freq
Street,2930,2,Pave,2918
Alley,198,2,Grvl,120
Central.Air,2930,2,Y,2734
Land.Contour,2930,4,Lvl,2633
Mas.Vnr.Type,1155,4,BrkFace,880
Misc.Feature,106,5,Shed,95
Lot.Config,2930,5,Inside,2140
Bldg.Type,2930,5,1Fam,2425
Roof.Style,2930,6,Gable,2321
Garage.Type,2773,6,Attchd,1731


Por fim, o código seleciona apenas as colunas numéricas do dataset e ordena as linhas em ordem crescente pelo número de valores não nulos.

In [23]:
data \
    .select_dtypes('number') \
    .describe() \
    .transpose() \
    .sort_values(by='count', ascending=True)

,count,mean,std,min,25%,50%,75%,max
Lot.Frontage,2440.0,6.922459e+01,2.336533e+01,21.0,5.800000e+01,68.0,8.000000e+01,3.130000e+02
Garage.Yr.Blt,2771.0,1.978132e+03,2.552841e+01,1895.0,1.960000e+03,1979.0,2.002000e+03,2.207000e+03
Mas.Vnr.Area,2907.0,1.018968e+02,1.791126e+02,0.0,0.000000e+00,0.0,1.640000e+02,1.600000e+03
Bsmt.Full.Bath,2928.0,4.313525e-01,5.248202e-01,0.0,0.000000e+00,0.0,1.000000e+00,3.000000e+00
Bsmt.Half.Bath,2928.0,6.113388e-02,2.452536e-01,0.0,0.000000e+00,0.0,0.000000e+00,2.000000e+00
Bsmt.Unf.SF,2929.0,5.592625e+02,4.394942e+02,0.0,2.190000e+02,466.0,8.020000e+02,2.336000e+03
BsmtFin.SF.1,2929.0,4.426296e+02,4.555908e+02,0.0,0.000000e+00,370.0,7.340000e+02,5.644000e+03
BsmtFin.SF.2,2929.0,4.972243e+01,1.691685e+02,0.0,0.000000e+00,0.0,0.000000e+00,1.526000e+03
Garage.Cars,2929.0,1.766815e+00,7.605664e-01,0.0,1.000000e+00,2.0,2.000000e+00,5.000000e+00
Total.Bsmt.SF,2929.0,1.051615e+03,4.406151e+02,0.0,7.930000e+02,990.0,1.302000e+03,6.110000e+03


Por último, vamos salvar o dataset com os tipos corretos como um `pickle` para ser utilizado posteriormente.

In [27]:
processed_dir = DATA_DIR / 'processed'
processed_dir.mkdir(parents=True, exist_ok=True)

In [29]:
processed_file_path = processed_dir / 'ames_with_correct_types.pkl'
with open(processed_file_path, 'wb') as file:
    pickle.dump(
        [
            data,
            continuous_variables,
            discrete_variables,
            ordinal_variables,
            categorical_variables,
        ],
        file,
    )